In [1]:
import os
os.chdir('/home/nmduy/CA683/Credit Project')
#from fancyimpute import IterativeImputer as MICE
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score 
import random


## READ DATA AND REMOVE NA
Read CSV and only keep the full-info cells

In [82]:
data = pd.read_csv('data/imputed_train.csv')
data_remove_id = data.drop(['SK_ID_CURR'], axis=1) if 'SK_ID_CURR' in list(data.columns) else data.copy()
data_remove_id = data_remove_id.drop(['DAYS_EMPLOYED_ANOM'], axis=1) if 'DAYS_EMPLOYED_ANOM' in list(data_remove_id.columns) else data_remove_id.copy()
print(data_remove_id.shape)
data_remove_id = data_remove_id.dropna()
data_remove_id = data_remove_id.reset_index(drop=True)
print(data_remove_id.shape)

(307511, 110)
(307511, 110)


## MAKE IT BALANCE
Create subset and maybe make it balance between classes within classes

In [83]:
##### MAKE IT BALANCE #####
random.seed(1509)

numb_pos_samples = data_remove_id['TARGET'].sum()
numb_neg_samples = len(data_remove_id) - numb_pos_samples
portion_pos_train = 0.8
portion_pos_val = 0.1
numb_pos_train = int(numb_pos_samples * portion_pos_train)
numb_pos_val = int(numb_pos_samples * portion_pos_val)
numb_pos_test = numb_pos_samples - numb_pos_train - numb_pos_val

index_pos = data_remove_id.index[data_remove_id['TARGET'] == 1].tolist()
index_neg = [i for i in range(len(data_remove_id)) if i not in index_pos]

data_pos = data_remove_id.iloc[np.asarray(index_pos), :]
data_neg = data_remove_id.iloc[np.asarray(index_neg), :]

##### shuffle #####
data_pos = data_pos.sample(frac=1)
data_neg = data_neg.sample(frac=1)

data_pos_train = data_pos.iloc[0:numb_pos_train,:]
data_pos_val = data_pos.iloc[numb_pos_train:numb_pos_train + numb_pos_val, :]
data_pos_test = data_pos.iloc[numb_pos_train + numb_pos_val:, :]

data_neg_train = data_neg.iloc[0:int(numb_pos_train*11), :]
data_neg_val = data_neg.iloc[int(numb_pos_train*11):-numb_pos_test, :]
data_neg_test = data_neg.iloc[-numb_pos_test:, :]

data_train = pd.concat([data_pos_train, data_neg_train], ignore_index=True)
data_val = pd.concat([data_pos_val, data_neg_val], ignore_index=True)
data_not_test = pd.concat([data_train, data_val], ignore_index=True)
data_test = pd.concat([data_pos_test, data_neg_test], ignore_index=True)

print(f"Numb train/val/test samples: {len(data_train)} / {len(data_val)} / {len(data_test)}")

print(data_train.shape)

Numb train/val/test samples: 238320 / 64225 / 4966
(238320, 110)


## REMOVE FT CONTAIN ONLY 1 VALUES
Sometimes the binary features contains only 1 value due to the dropna functions

In [84]:
##### REMOVE 1-VALUE FT #####
remove_ft = []
for col in list(data_not_test.drop('TARGET', axis=1).columns):
    if data_not_test[col].sum() == 0 or data_not_test[col].sum() == len(data_not_test[col]):
        remove_ft += [col]

print(remove_ft)        
print(len(remove_ft))

data_train = data_train.drop(remove_ft, axis=1)
data_val = data_val.drop(remove_ft, axis=1)
data_test = data_test.drop(remove_ft, axis=1)
data_not_test = data_not_test.drop(remove_ft, axis=1)

[]
0


## REMOVE COLLINEAR
Some of ft still have a strong relationship with others --> Need to remove one of them

##### FIND CORRELATION MATRIX #####
corr_matrix = data_not_test.corr()
corr_matrix = corr_matrix.replace(1, 0)

corr_max = corr_matrix.max(axis = 0)
corr_min = corr_matrix.min(axis = 0)
corr_max_idx = corr_matrix.idxmax(axis = 0)
corr_min_idx = corr_matrix.idxmin(axis = 0)

dict_corr_max = dict(corr_max)
dict_corr_max_idx = dict(corr_max_idx)
dict_corr_max_filter = {}
for x in dict_corr_max.keys():
    if dict_corr_max[x] > 0.8:
        dict_corr_max_filter[x] = dict_corr_max[x]
dict_corr_max_idx_filter = {}
for x in dict_corr_max_filter.keys():
   dict_corr_max_idx_filter[x] = dict_corr_max_idx[x]

dict_corr_min = dict(corr_min)
dict_corr_min_idx = dict(corr_min_idx)
dict_corr_min_filter = {}
for x in dict_corr_min.keys():
    if dict_corr_min[x] < -0.8:
        dict_corr_min_filter[x] = dict_corr_min[x]
dict_corr_min_idx_filter = {}
for x in dict_corr_min_filter.keys():
   dict_corr_min_idx_filter[x] = dict_corr_min_idx[x]


dict_corr_max_filter

dict_corr_min_filter

In [9]:
##### REMOVE COLLINEAR ####
# remove_ft = ['AMT_GOODS_PRICE', 'REGION_RATING_CLIENT_W_CITY', 'OBS_60_CNT_SOCIAL_CIRCLE', 'CNT_CHILDREN', 
#              'LIVE_REGION_NOT_WORK_REGION', 'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Secondary / secondary special']
# remove_ft = ['LIVE_REGION_NOT_WORK_REGION', 'DEF_60_CNT_SOCIAL_CIRCLE']
# remove_ft = ['AMT_GOODS_PRICE', 'REGION_RATING_CLIENT_W_CITY', 'OBS_60_CNT_SOCIAL_CIRCLE']
remove_ft = []
if len(remove_ft) > 0:
    data_train = data_train.drop(remove_ft, axis=1)
    data_test = data_test.drop(remove_ft, axis=1)
    data_val = data_val.drop(remove_ft, axis=1)
    data_not_test = data_not_test.drop(remove_ft, axis=1)

data_train.shape

(26872, 88)

## REMOVE DOMINANT FEATURES
Dominant features are features containing almost only 1 values in entire dataset --> SHOULD REMOVE IT

In [ ]:
categorical_name = ['NAME_CONTRACT_TYPE', 'CODE_GENDER']
pc_pos = {}
for col in list(data_not_test.columns):
    if 'NAME' in col or 'OCCUPATION' in col or 'WEEKDAY' in col or 'ORGANIZATION' in col:
        categorical_name += [col]
    if col in categorical_name:
        values = np.asarray(list(data_remove_id[col]))
        pc_pos[col] = np.sum(values) / len(values)
remove_ft_dominant = []
for col in categorical_name:
    if pc_pos[col] < 0.01 or pc_pos[col] > 0.99:
        remove_ft_dominant += [col]
        
print(len(remove_ft_dominant))

##### REMOVE DOMINANT FT IN BINARY #####
data_train = data_train.drop(remove_ft_dominant, axis=1)
data_val = data_val.drop(remove_ft_dominant, axis=1)
data_test = data_test.drop(remove_ft_dominant, axis=1)
data_not_test = data_not_test.drop(remove_ft_dominant, axis=1)

print(data_train.shape)

## TRAIN

In [85]:
X_train = data_train.drop(['TARGET'], axis = 1)
X_test = data_test.drop(['TARGET'], axis = 1)
X_val = data_val.drop(['TARGET'], axis = 1)
y_train = data_train['TARGET']
y_test = data_test['TARGET']
y_val = data_val['TARGET']
X_not_test = data_not_test.drop(['TARGET'], axis = 1)
y_not_test = data_not_test['TARGET']
print(X_train.shape)

(238320, 109)


In [213]:
logmodel = LogisticRegression(C=1e9, max_iter=500, class_weight={0:1, 1:12})
logmodel.fit(X_train, y_train)

LogisticRegression(C=1000000000.0, class_weight={0: 1, 1: 12}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## EVALUATION

In [214]:
def calculate_metric(gtnp, pdnp):
    # input are numpy vector
    total_samples = len(gtnp)
    #print(f"Total sample: {total_samples}")
    total_correct = np.sum(gtnp == pdnp)
    accuracy = total_correct / total_samples
    gt_pos = np.where(gtnp == 1)[0]
    gt_neg = np.where(gtnp == 0)[0]
    TP = np.sum(pdnp[gt_pos])
    TN = np.sum(1 - pdnp[gt_neg])
    FP = np.sum(pdnp[gt_neg])
    FN = np.sum(1 - pdnp[gt_pos])
    precision = TP / (TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*precision*recall/(precision+recall)
    metrics = {}
    metrics['accuracy'] = str(accuracy)
    metrics['precision'] = str(precision)
    metrics['recall'] = str(recall)
    metrics['f1'] = str(f1)
    metrics['tp'] = str(int(TP))
    metrics['tn'] = str(int(TN))
    metrics['fp'] = str(int(FP))
    metrics['fn'] = str(int(FN))

    return metrics

In [215]:
##### TRAIN PREDICTION #####
train_predict = logmodel.predict(X_train)
metric = calculate_metric(np.asarray(list(y_train)), train_predict)
metric['auc'] = roc_auc_score(y_train,train_predict)
metric

{'accuracy': '0.5412932192010742',
 'precision': '0.1037394023689083',
 'recall': '0.5896273917421954',
 'f1': '0.17643646554516756',
 'tp': '11710',
 'tn': '117291',
 'fp': '101169',
 'fn': '8150',
 'auc': 0.5632632976288565}

In [216]:
##### VAL PREDICTION #####
val_predict = logmodel.predict(X_val)
metric = calculate_metric(np.asarray(list(y_val)), val_predict)
metric['auc'] = roc_auc_score(y_val,val_predict)
metric

{'accuracy': '0.5383261969637991',
 'precision': '0.04813225559658051',
 'recall': '0.5829975825946817',
 'f1': '0.08892302965125211',
 'tp': '1447',
 'tn': '33127',
 'fp': '28616',
 'fn': '1035',
 'auc': 0.5597640197442902}

In [217]:
##### TEST PREDICTION #####
test_predict = logmodel.predict(X_test)
metric = calculate_metric(np.asarray(list(y_test)), test_predict)
metric['auc'] = roc_auc_score(y_test, test_predict)
metric

{'accuracy': '0.5712847362062021',
 'precision': '0.56855151045701',
 'recall': '0.5912202980265807',
 'f1': '0.5796643632773938',
 'tp': '1468',
 'tn': '1369',
 'fp': '1114',
 'fn': '1015',
 'auc': 0.5712847362062021}

## IMPROVE BY ADDING BIAS TERM
Assume that we know the portion of positive in training and population, we can adjust the bias term and achieve higher performance (if possible)

In [ ]:
origin_intercept = logmodel.intercept_

In [136]:
##### ADJUST BIAS TERM #####
pop_portion = 0.5
sample_portion = 0.02 # if training data is balance


In [137]:
logmodel.intercept_ = origin_intercept - np.log(((1-pop_portion)/pop_portion) * (sample_portion/(1-sample_portion)))

In [138]:
logmodel.intercept_

array([3.89182052])

In [139]:
##### VAL PREDICTION #####
val_predict = logmodel.predict(X_val)
metric = calculate_metric(np.asarray(list(y_val)), val_predict)
metric

{'accuracy': '0.009826988809289694',
 'precision': '0.009826988809289694',
 'recall': '1.0',
 'f1': '0.019462717709450256',
 'tp': '1679',
 'tn': '0',
 'fp': '169177',
 'fn': '0'}